In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import get_window

def gated_fourier_analysis(signal, window_size, hop_size, window_type='hann', fs=1.0, plot=False):
    """
    Perform Gated (Windowed) Fourier Analysis (STFT) on a 1D signal.

    Parameters:
    - signal (array-like): Input signal
    - window_size (int): Size of the analysis window
    - hop_size (int): Step size between successive windows
    - window_type (str): Type of window (e.g. 'hann', 'gaussian', 'hamming')
    - fs (float): Sampling frequency (for frequency axis)
    - plot (bool): If True, show the spectrogram

    Returns:
    - stft_matrix (2D np.array): Complex STFT matrix [freq, time]
    - freqs (1D np.array): Frequency axis
    - times (1D np.array): Time axis
    """

    signal = np.asarray(signal)
    window = get_window(window_type, window_size, fftbins=True)
    num_frames = 1 + (len(signal) - window_size) // hop_size
    stft_matrix = np.zeros((window_size // 2 + 1, num_frames), dtype=np.complex64)

    for i in range(num_frames):
        start = i * hop_size
        segment = signal[start:start + window_size]
        if len(segment) < window_size:
            segment = np.pad(segment, (0, window_size - len(segment)), mode='constant')

        windowed = segment * window
        spectrum = np.fft.rfft(windowed)
        stft_matrix[:, i] = spectrum

    freqs = np.fft.rfftfreq(window_size, d=1/fs)
    times = np.arange(num_frames) * hop_size / fs

    if plot:
        plt.figure(figsize=(10, 5))
        plt.pcolormesh(times, freqs, 20 * np.log10(np.abs(stft_matrix) + 1e-10), shading='gouraud')
        plt.title('Spectrogram (Gated Fourier Analysis)')
        plt.xlabel('Time [s]')
        plt.ylabel('Frequency [Hz]')
        plt.colorbar(label='Magnitude [dB]')
        plt.tight_layout()
        plt.show()

    return stft_matrix, freqs, times


In [ ]:
import numpy as np

# Example signal: 2 sinusoids
fs = 1000  # Sampling rate in Hz
t = np.linspace(0, 1, fs, endpoint=False)
signal = np.sin(2 * np.pi * 50 * t) + np.sin(2 * np.pi * 120 * t)

# Run gated Fourier analysis
stft_matrix, freqs, times = gated_fourier_analysis(signal, window_size=256, hop_size=128, fs=fs, plot=True)
